In [1]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001.parquet', 'validation': 'data/validation-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/allenai/sciq/" + splits["train"])
df

,question,distractor3,distractor1,distractor2,correct_answer,support
0,What type of organism is commonly used in prep...,viruses,protozoa,gymnosperms,mesophilic organisms,"Mesophiles grow best in moderate temperature, ..."
1,What phenomenon makes global winds blow northe...,tropical effect,muon effect,centrifugal effect,coriolis effect,Without Coriolis Effect the global winds would...
2,Changes from a less-ordered state to a more-or...,endothermic,unbalanced,reactive,exothermic,Summary Changes of state are examples of phase...
3,What is the least dangerous radioactive decay?,zeta decay,beta decay,gamma decay,alpha decay,All radioactive decay is dangerous to living t...
4,Kilauea in hawaii is the world’s most continuo...,magma,greenhouse gases,carbon and smog,smoke and ash,Example 3.5 Calculating Projectile Motion: Hot...
...,...,...,...,...,...,...
11674,The enzyme pepsin plays an important role in t...,lipids,protons,proteins,peptides,Protein A large part of protein digestion take...
11675,What remains a constant of radioactive substan...,acidity,temperature,volatility,rate of decay,The rate of decay of a radioactive substance i...
11676,"Terrestrial ecosystems, also known for their d...",substrates,bisomes,monomes,biomes,"Terrestrial ecosystems, also known for their d..."
11677,High explosives create shock waves that exceed...,turbulence,light speed,ion speed,supersonic,The modern day formulation of gun powder is ca...


In [2]:
df.loc[0]

question          What type of organism is commonly used in prep...
distractor3                                                 viruses
distractor1                                                protozoa
distractor2                                             gymnosperms
correct_answer                                 mesophilic organisms
support           Mesophiles grow best in moderate temperature, ...
Name: 0, dtype: object

In [3]:
import torch
import os

from sentence_transformers import CrossEncoder
from src.chain_initialisation import init_chain
from src.embedding_management import init_embeddings, add_embeddings_from_files
from src.model_initialisation import init_model
from src.pipeline_initialisation import init_pipeline
from src.retriever_initialisation import init_retriever
from src.dynamic_doc_retrieval import download_documents, initialise_keyword_model, generate_query_from_question

2025-05-28 14:20:32.188255: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748434832.594573   33617 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748434832.634021   33617 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748434832.751645   33617 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748434832.751715   33617 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748434832.751724   33617 computation_placer.cc:177] computation placer alr

In [5]:
BASE_DIR = os.curdir
DATA_DIR = os.path.join(BASE_DIR, "data")
DYNAMIC_DATA_DIR = os.path.join(DATA_DIR, "dynamic")
print(f"dynamic data dir: {DYNAMIC_DATA_DIR}")


def rerank_documents(reranker, query: str, documents: list, top_k: int = 3) -> list:
    if not documents:
        return []

    pairs = [(query, doc.page_content) for doc in documents]
    scores = reranker.predict(pairs)
    ranked = sorted(zip(documents, scores), key=lambda x: x[1], reverse=True)

    return [doc for doc, score in ranked[:top_k] if score > 0.5]

def ask_question(model_chain, reranker, keyword_model, vectorstore, question, num_docs, max_tries, online, rerank_top_k):
    if online:
        print("Obtaining data from scholar...")
        query = generate_query_from_question(keyword_model, question)
        downloaded_files = download_documents(query, DYNAMIC_DATA_DIR, num_docs, max_tries)
        add_embeddings_from_files(vectorstore, downloaded_files)

    response = model_chain.invoke({
        "question": question,
        "chat_history": []
    })
    answer = response["answer"].split("### Answer:")[-1].strip()
    source_documents = response["source_documents"]
    filtered_docs = rerank_documents(reranker, question, source_documents, top_k=rerank_top_k)
    return answer, source_documents, filtered_docs

dynamic data dir: ./data/dynamic


In [4]:
cuda_available = torch.cuda.is_available()
print(f"Initializing model... CUDA available: {cuda_available}")
model, model_name = init_model(cuda_available)
pipeline = init_pipeline(model=model, model_name=model_name)
vectorstore = init_embeddings(cuda_available)
retriever = init_retriever(vectorstore)
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
chain = init_chain(pipeline=pipeline, retriever=retriever)
keyword_model = initialise_keyword_model()
print("Start-up complete.")

Initializing model... CUDA available: False


Device set to use cpu
/home/henri/ul-fri-nlp-course-project-2024-2025-drop-table-teams/src/embedding_management.py:17: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(


Loading FAISS from file - skipping embedding build.
Vectorstore loaded successfully.
Retriever initialized with k=2 and similarity threshold=0.8


/home/henri/ul-fri-nlp-course-project-2024-2025-drop-table-teams/src/chain_initialisation.py:7: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipeline)


LLM model chain created.
Start-up complete.


In [13]:
question = df.loc[0]["question"]
answer, source_docs = ask_question(question, online=False)
answer

/home/henri/.local/lib/python3.10/site-packages/langchain_core/vectorstores/base.py:1082: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='7e08dabe-c118-4506-8c9e-d911c1835182', metadata={'source': '/home/henri/ul-fri-nlp-course-project-2024-2025-drop-table-teams/data/fao780/fao780/docsutf8/t5816e.txt'}, page_content='A list of species produced in the region is given in Table 5.\n2.3 Production systems and practices in the region\nA diversity of species is cultured throughout the region in relatively small volumes. Consequently a wide variety of aquaculture practices are used, ranging from intensive and semi-intensive systems for producing salmonids and marine shrimp, to extensive systems for the culture of fish, giant clams, seaweed, and pearl oysters.'), np.float32(-24.619614)), (Document(id='2ee5e358-a0ea-4f13-b6d0-56c34630a209', metadata={'source': '/home/henri/ul-fri-nlp-course-project-2024-2025-drop-table-teams/data/fao780/fao780/docsutf8/x0209e.txt'}, pa

"The following two animals-- fish (from which milk may also come) and other fishes--- include an invertebrate named Megaptera klassensis although its genus does not exist at present there.(Tasmanian Sea Turtles.) These species live around 70 percent of the ocean basins covering about 65 percent water(Suez Formation Water Source). Aquamarine waters typically contain less than five feet between three degrees south latitude=0 deg C..ocean surface temperature< 30 °C > below sea level range on both ends of the main boundary line during estuaries up to 4° s(-1 ) above 100 yz...[p]atoll ice thickness < 0.5 mm−2; maximum energy needed per unit volume ~ 20 Mg((m*r+8)/4 ); most bodies form into four'submersible' submaploidic marine mounds 2 km² apart including cetaceans plus halos.[i]The vast majority rely upon saltwater derived material after several thousand years [and more significantly provide dietary nutrients]. In some instances their diets consist primarily comprised sugar based meals con

In [6]:
from sentence_transformers import SentenceTransformer, util
from fuzzywuzzy import fuzz

# Initialize embedding model
evaluation_model = SentenceTransformer('all-MiniLM-L6-v2')

In [14]:
def evaluate_answer(row):
    question = row['question']
    correct_answer = row['correct_answer']
    support = row['support']
    distractors = [row['distractor1'], row['distractor2'], row['distractor3']]

    # Chatbot answer
    response, source_docs, filtered_docs = ask_question(chain, reranker, keyword_model, vectorstore, question, 2, 5, True, 5)
    response = response.lower()

    # Exact match
    exact = int(correct_answer.lower() in response)

    # Fuzzy match
    fuzzy_score = fuzz.partial_ratio(correct_answer.lower(), response)

    # Embedding similarity to correct answer
    emb_correct = evaluation_model.encode(correct_answer, convert_to_tensor=True)
    emb_response = evaluation_model.encode(response, convert_to_tensor=True)
    sim_correct = util.cos_sim(emb_correct, emb_response).item()

    # Embedding similarity to support passage
    emb_support = evaluation_model.encode(support, convert_to_tensor=True)
    sim_support = util.cos_sim(emb_support, emb_response).item()

    # Compare similarity to distractors
    sim_distractors = []
    for d in distractors:
        emb_distractor = evaluation_model.encode(d, convert_to_tensor=True)
        sim_distractors.append(util.cos_sim(emb_distractor, emb_response).item())
    
    # Is chatbot closer to correct than to all distractors?
    correct_beats_distractors = int(sim_correct > max(sim_distractors))

    return {
        "question": question,
        "chatbot_response": response,
        "exact_match": exact,
        "fuzzy_score": fuzzy_score,
        "sim_to_correct": round(sim_correct, 3),
        "sim_to_support": round(sim_support, 3),
        "correct_beats_distractors": correct_beats_distractors,
        "source_docs": source_docs,
        "filtered_docs": filtered_docs
    }


In [15]:
df.loc[:3]

,question,distractor3,distractor1,distractor2,correct_answer,support
0,What type of organism is commonly used in prep...,viruses,protozoa,gymnosperms,mesophilic organisms,"Mesophiles grow best in moderate temperature, ..."
1,What phenomenon makes global winds blow northe...,tropical effect,muon effect,centrifugal effect,coriolis effect,Without Coriolis Effect the global winds would...
2,Changes from a less-ordered state to a more-or...,endothermic,unbalanced,reactive,exothermic,Summary Changes of state are examples of phase...
3,What is the least dangerous radioactive decay?,zeta decay,beta decay,gamma decay,alpha decay,All radioactive decay is dangerous to living t...


In [ ]:
title = "eval_results"

results = [evaluate_answer(row) for _, row in df.loc[:3].iterrows()]
results_df = pd.DataFrame(results)

Obtaining data from scholar...
Querying Google Scholar for: yogurt foods cheese organism preparation


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'analyticalsciencejournals.onlinelibrary.wiley.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
[INFO] | 2025/05/28 14:56:08 | Choose scihub url [0]: https://sci-hub.se
[INFO] | 2025/05/28 14:56:09 | <- Request: scihub_url=https://sci-hub.se, source=DoiSource[type=doi, id=analyticalsciencejournals.onlinelibrary.wiley.com/doi/abs/10.1002/elsc.201100122], proxies={'http': 'socks5://127.0.0.1:7890'}



Result 1: Potential effects of probiotics in cheese and yogurt production: A review
Publication URL: https://analyticalsciencejournals.onlinelibrary.wiley.com/doi/abs/10.1002/elsc.201100122
Trying to download PDF from: https://analyticalsciencejournals.onlinelibrary.wiley.com/doi/full/10.1002/elsc.201100122
The eprint URL is not a direct link to a PDF.


[INFO] | 2025/05/28 14:56:09 | -> Response: status_code=200, content_length=8599
[INFO] | 2025/05/28 14:56:09 | * Extracted information: {'url': 'https://moscow.sci-hub.se/1161/d6aca80dff6715aa87a433761e628492/plessas2012.pdf', 'title': 'Potential effects of probiotics in cheese and yogurt production  A review. Engineering in Life Sciences, 12(4), 433–440'}
[INFO] | 2025/05/28 14:56:10 | ↓ Successfully download the url to: ./data/dynamic/Potential_effects_of_probiotics_in_cheese_and_yogurt_production_A_review.pdf
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.sciencedirect.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Successfully downloaded to ./data/dynamic/Potential_effects_of_probiotics_in_cheese_and_yogurt_production_A_review.pdf

Result 2: Inhibitory effects of Microgard™ on yogurt and cottage cheese spoilage organisms
Publication URL: https://www.sciencedirect.com/science/article/pii/S0022030290787449
Trying to download PDF from: https://www.sciencedirect.com/science/article/pii/S0022030290787449/pdf?md5=271e01ddf92d493dc06ad431140c96c9&pid=1-s2.0-S0022030290787449-main.pdf
The eprint URL is not a direct link to a PDF.


...

[INFO] | 2025/05/28 14:59:02 | -> Response: status_code=403, content_length=898
[WARNING] | 2025/05/28 14:59:02 | Error occurs, task status: crawling_failed, error: Error occurs when crawling source: DoiSource[type=doi, id=link.springer.com/chapter/10.1007/978-3-642-20186-8_11]
[INFO] | 2025/05/28 14:59:02 | Choose scihub url [1]: https://sci-hub.ru
[INFO] | 2025/05/28 14:59:02 | <- Request: scihub_url=https://sci-hub.ru, source=DoiSource[type=doi, id=link.springer.com/chapter/10.1007/978-3-642-20186-8_11], proxies={'http': 'socks5://127.0.0.1:7890'}
[INFO] | 2025/05/28 14:59:02 | -> Response: status_code=403, content_length=898
[WARNING] | 2025/05/28 14:59:02 | Error occurs, task status: crawling_failed, error: Error occurs when crawling source: DoiSource[type=doi, id=link.springer.com/chapter/10.1007/978-3-642-20186-8_11]
[INFO] | 2025/05/28 14:59:02 | Choose scihub url [2]: https://sci-hub.in
[INFO] | 2025/05/28 14:59:02 | <- Request: scihub_url=https://sci-hub.in, source=DoiSource[

Successfully downloaded to ./data/dynamic/Modes_of_radioactive_decay.pdf


/home/henri/.local/lib/python3.10/site-packages/langchain_core/vectorstores/base.py:1082: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='e777ad1d-2e2d-4a62-a543-d377323334d7', metadata={'source': './data/dynamic/Modeling_radioactive_decay.pdf'}, page_content='Keywords : Half-mean life, modeling, radioactive decay, radioactivity teaching, simulation \nI. Introduction  \nRadioactive decay is the name given to the natural process by which an atomic nucleus spontaneously transforms \nitself (decays) into another, releasing dangerous high energy emissions (Hughes and Zalts, 2000). \nIf a sample consists of N0 radioactive nuclei at time t=0, and if N(t) denotes the number of the nuclei remaining at'), np.float32(-18.21845)), (Document(id='31e86bba-6e51-4ad3-978d-85236a18f92b', metadata={'source': './data/dynamic/Modeling_radioactive_decay.pdf'}, page_content='Keywords : Half-mean life, modeling, radioactive decay, radioactivity teaching, simulation \nI. Introductio

Vectorstore updated with 426 document splits.


FileExistsError: [Errno 17] File exists: 'evaluation'

In [17]:
if not os.path.exists("evaluation"):
    os.makedirs("evaluation")
results_df.to_csv(f"evaluation/{title}.csv", index=False)
results_df

,question,chatbot_response,exact_match,fuzzy_score,sim_to_correct,sim_to_support,correct_beats_distractors,source_docs,filtered_docs
0,What type of organism is commonly used in prep...,-------------------- -------- ---------- -----...,0,45,0.294,0.214,1,[],[]
1,What phenomenon makes global winds blow northe...,(t)he greatest hurricane of all times would ha...,0,60,0.101,0.194,0,[],[]
2,Changes from a less-ordered state to a more-or...,you can only switch between different states w...,0,50,0.006,0.207,0,[],[]
3,What is the least dangerous radioactive decay?,- there can be less than 0 contamination betwe...,0,36,0.160,0.445,1,[],[]
